In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML, display

In [2]:
# Upload de Datasets
# https://drive.google.com/drive/folders/1oncC_yOvZeNGmC_hLJeavdM4MPq3BkNd
# deben estar en la carpeta original_datasets

#users_ff = pd.read_csv('original_datasets/users_ff.csv')   -- No se incluye en el modelo 
users_ss1 = pd.read_csv('original_datasets/users_ss1.csv')
users_ss2 = pd.read_csv('original_datasets/users_ss2.csv')
users_ss3 = pd.read_csv('original_datasets/users_ss3.csv')
#users_ts = pd.read_csv('original_datasets/users_ts.csv')
users_humans = pd.read_csv('original_datasets/users_gen.csv')

# tweets_ff = pd.read_csv('original_datasets/tweets_ff.csv')   -- No se incluye en el modelo 
tweets_ss1 = pd.read_csv('original_datasets/tweets_ss1.csv')
tweets_ss2 = pd.read_csv('original_datasets/tweets_ss2.csv')
tweets_ss3 = pd.read_csv('original_datasets/tweets_ss3.csv')
#tweets_ts = pd.read_csv('original_datasets/tweets_ts.csv')
tweets_humans = pd.read_csv('original_datasets/tweets_gen.csv')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preparación de Datasets

In [3]:
# Asignar labels
users_ss1.loc[:,'Cat'] = 'SocialSpambot1'
users_ss2.loc[:,'Cat'] = 'SocialSpambot2'
users_ss3.loc[:,'Cat'] = 'SocialSpambot3'
#users_ts.loc[:,'Cat'] = 'TraditionalSpambot'
#users_ff.loc[:,'Cat'] = 'FollowersBot'   -- No se incluye en el modelo
users_humans.loc[:,'Cat'] = 'Humans'
tweets_ss1['Cat'] = 'SocialSpambot1'
tweets_ss2['Cat'] = 'SocialSpambot2'
tweets_ss3['Cat'] = 'SocialSpambot3'
#tweets_ts['Cat'] = 'TraditionalSpambot'
#tweets_ff['Cat'] = 'FollowersBot'   -- No se incluye en el modelo
tweets_humans['Cat'] = 'Humans'

# Crear label binaria 'bot'
users_ss1.loc[:,'bot'] = True
users_ss2.loc[:,'bot'] = True
users_ss3.loc[:,'bot'] = True
#users_ts.loc[:,'bot'] = True
users_humans.loc[:,'bot'] = False
tweets_ss1.loc[:,'bot'] = True
tweets_ss2.loc[:,'bot'] = True
tweets_ss3.loc[:,'bot'] = True
#tweets_ts.loc[:,'bot'] = True
tweets_humans.loc[:,'bot'] = False

# Concatenar datasets de USUARIOS (SocialSpambots & Traditional Spambots)
users = pd.concat([users_ss1,users_ss2,users_ss3,users_humans],ignore_index=True)

# Concatenar datasets de TWEETS (SocialSpambots & Traditional Spambots)
tweets = pd.concat([tweets_ss1,tweets_ss2,tweets_ss3,tweets_humans],ignore_index=True)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



### Exploración

In [4]:
# Identificar columnas que no coinciden entre datasets
total_columns = []
for x in [users_ss1, users_ss2, users_ss3, users_humans]:
#     print(x.columns.values)
    total_columns = np.concatenate((total_columns, x.columns.values))
    
total_columns = np.unique(np.array(total_columns))
dfHasColumns = pd.DataFrame( columns = total_columns)
for dfName in ['users_ss1', 'users_ss2', 'users_ss3', 'users_humans']:
    dataFrame = globals()[dfName]
    cols = dataFrame.columns.values
    hasCol = [1 if x in(cols) else 0 for x in dfHasColumns.columns.values]
    dfHasColumns.loc[dfName] = hasCol
dfHasColumns  = dfHasColumns.T
dfHasColumns.loc[:,'total'] = dfHasColumns.sum(axis=1)

dfHasColumns.sort_values(by='total').head(7)

,users_ss1,users_ss2,users_ss3,users_humans,total
test_set_2,0,0,1,1,2.0
test_set_1,1,0,0,1,2.0
Cat,1,1,1,1,4.0
profile_background_tile,1,1,1,1,4.0
profile_banner_url,1,1,1,1,4.0
profile_image_url,1,1,1,1,4.0
profile_image_url_https,1,1,1,1,4.0


#### Conclusiones de esta tabla:
La columna 'crawled_at' no se encuentra en users_ff (Folowers Bot).<br>
Esta columna es importante debido a que nos da una idea de la cantidad de meses que ese bot estuvo activo (y nos permite calcular una vida promedio del bot)
Hay tres opciones:<br>
1 No calcular la cantidad de meses antes que un bot se crea.<br>
2 No utilizar ese dataset.<br>
3 Asumir la fecha updated que esta presente en todos los casos

In [10]:
# Promedios de fecha de recolección por categoria
a = users[['updated','Cat']]
a.updated = pd.to_datetime(a.updated).astype(int)
b = a.groupby('Cat').mean()
b.updated = pd.to_datetime(b.updated)
b

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [int32]

In [7]:
pd.to_datetime(pd.to_datetime(users[users.Cat=='SocialSpambot']['crawled_at']).astype(int).mean())

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [int32]

In [8]:
# USERS Promedios de fecha de Recolección por Categoria
print('SocialSpambot     : ',pd.to_datetime(pd.to_datetime(users[users.Cat=='SocialSpambot']['crawled_at']).astype(int).mean()))
print('Humans            : ',pd.to_datetime(pd.to_datetime(users[users.Cat=='Humans']['crawled_at']).astype(int).mean()))

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [int32]

In [ ]:
# Promedios de fecha de recolección por categoria
tweets.crawled_at = pd.to_datetime(tweets.crawled_at,errors='coerce')
a = tweets[['crawled_at','Cat']]
a.crawled_at = a.crawled_at.astype(int)
b = a.groupby('Cat').mean()
b.crawled_at = pd.to_datetime(b.crawled_at)
b

In [ ]:
display(tweets_humans.columns)

first = True
def getDataTweets(twU):
    global first
    if(first):
        dates = pd.to_datetime(twU.timestamp) #aca lo que hago es guardar la columnoa como datetime
        twU = twU.sort_values(by='timestamp', ascending=0)
        twU.loc[:,'dayOfWeek'] = dates.dt.dayofweek # dia de la semana
        twU.loc[:,'hour'] = dates.dt.hour #hora del tweet
        twU.loc[:,'weekNum'] = dates.apply(lambda x: str(x.week) + '-' + str(x.year))
        display(twU.head())
        
        times = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp) # calculo distancia entre tweets
        display(times.median())
    first = False

tweets_humans.groupby('user_id').apply(getDataTweets)

In [ ]:
print('Usuarios unicos en dataset de Users: ',users.id.nunique())
print('Usuarios unicos en dataset de Tweets: ',tweets.user_id.nunique())

In [ ]:
# Crear tabla de cuenta de tweets desde dataset de tweets
id_count_tweets = tweets[['user_id','id']].groupby('user_id',as_index=False).count()
id_count_tweets.rename(columns={'user_id':'id','id':'tweet_count'}, inplace=True)
# Joinear datasets para agregar a Users la cuenta tweets
users = users.merge(id_count_tweets,how='left',on='id')

In [ ]:
print('Cantidad de usuarios sin tweets: ',users.tweet_count.isna().sum())

In [ ]:
# Explorar usuarios sin tweets por categoria
users[['Cat','id']][users.tweet_count.isna()].groupby('Cat').count()

In [ ]:
print('Cantidad de usuarios Humanos únicos en el dataset original: ',users[users.Cat=="Humans"].id.nunique())
print('Proporcion de usuarios humanos que tienen tweets: ',tweets[tweets.Cat=="Humans"].user_id.nunique()/users[users.Cat=="Humans"].id.nunique())

### Limpieza Dataset USERS

In [11]:
# Eliminar columnas no utilizables
users.drop(columns=['test_set_1','test_set_2'],inplace=True)
# rectificar formato de fecha
users['crawled_at'] = pd.to_datetime(users['crawled_at'])
users['updated'] = pd.to_datetime(users['updated'])
# Procesamiento COLUMNAS
# default_profile_image
users.drop(columns='default_profile_image',inplace=True)  # solo 60 valores - se descarta
# default_profile
users.default_profile.fillna(0,inplace=True) # 1 = tema y background default de twitter   0 = modificado
# descriptiontweets.retweeted_status_id = tweets.retweeted_status_id.notna()  # Convertir a Dummy
users.description = users.description.notna()  # Convertir a Dummy 'tiene o no tiene descripción'
# eliminar columnas vacías
users.drop(columns=['contributors_enabled','follow_request_sent','following','is_translator','notifications','protected','verified'],inplace=True)
# Corregir formato timestamp
users.crawled_at = pd.to_datetime(users.crawled_at)
users.timestamp = pd.to_datetime(users.timestamp)
users.updated = pd.to_datetime(users.updated)

In [12]:
# Convertir en Dummy
users[['description','lang','location','geo_enabled','name','url','profile_banner_url',\
       'profile_use_background_image','profile_background_tile','profile_background_color',\
       'profile_background_image_url','screen_name','profile_text_color',\
       'profile_image_url_https','profile_link_color','profile_sidebar_border_color',\
       'profile_sidebar_fill_color','profile_image_url','profile_background_image_url_https']] = \
users[['description','lang','location','geo_enabled','name','url','profile_banner_url',\
       'profile_use_background_image','profile_background_tile','profile_background_color',\
       'profile_background_image_url','screen_name','profile_text_color',\
       'profile_image_url_https','profile_link_color','profile_sidebar_border_color',\
       'profile_sidebar_fill_color','profile_image_url','profile_background_image_url_https']].notna()

In [13]:
# Favourites Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'favs_tiempo'] = users.favourites_count / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='favourites_count',inplace=True)

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [int32]

In [ ]:
# Followers Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'followers_tiempo'] = users.followers_count / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='followers_count',inplace=True)

In [ ]:
# Statuses Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'statuses_tiempo'] = users.statuses_count / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='statuses_count',inplace=True)

In [ ]:
# Friends Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'friends_tiempo'] = users.friends_count / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='friends_count',inplace=True)

### Limpieza Dataset TWEETS

In [ ]:
# Eliminar columnas vacías o con muy pocos valores
tweets.drop(columns=['truncated','geo','contributors','favorited','retweeted','possibly_sensitive','place'],inplace=True)
# source
tweets.drop(columns=['source'],inplace=True)   # No utilizable
# in_reply_to_screen_name & in_reply_to_status_id 
tweets.drop(columns=['in_reply_to_screen_name'],inplace=True)   # nos quedamos con in reply user id
tweets.drop(columns=['in_reply_to_status_id'],inplace=True)   # nos quedamos con in reply user id
# in_reply_to_user_id
tweets.in_reply_to_user_id = tweets.in_reply_to_user_id.notna()  # Convertir a Dummy
# retweeted_status_id
tweets.retweeted_status_id = tweets.retweeted_status_id.notna()  # Convertir a Dummy
# Corregir formato timestamp
tweets.crawled_at = pd.to_datetime(tweets.crawled_at)
tweets.timestamp = pd.to_datetime(tweets.timestamp)
tweets.updated = pd.to_datetime(tweets.updated)

In [ ]:
# Nos quedamos solo con los usuarios que tienen tweets
users = users[users.id.isin(tweets.user_id.unique())]

In [ ]:
# Y con los tweets que correspondan a usuarios de la base
tweets = tweets[tweets.user_id.isin(users.id)]

In [ ]:
print('Cantidad de usuarios humanos válidos con tweets:',users.id[users.Cat=='Humans'].nunique())
print('Cantidad de tweets:',tweets.id[tweets.Cat=='Humans'].count())
print('Cantidad de usuarios bot válidos con tweets:',users.id[users.Cat=='SocialSpambot'].nunique())
print('Cantidad de tweets:',tweets.id[tweets.Cat=='SocialSpambot'].count())

In [ ]:
# Dump de ambos datastes to pickle
users.to_pickle('users.pkl')
# tweets.to_csv('tweets.csv') # OJO - Genera un archivo de casi 1 giga 